In [ ]:
!pip install kfp > /dev/null 2>&1

In [ ]:
import os
import kfp
import pandas as pd
from datetime import datetime
from kubernetes.client.models import V1EnvVar, V1SecretKeySelector

In [ ]:
# Notebooks for the pipeline
## generate-stats.ipynb
## generate-contributions.ipynb
## overview.ipynb

In [ ]:
url = 'http://yukked1.fyre.ibm.com:31380/pipeline'

# configures artifact location
notebook_location = kfp.dsl.ArtifactLocation.s3(
                            bucket="lresende",
                            endpoint="yukked1.fyre.ibm.com:31128",
                            insecure=True,
                            access_key_secret=V1SecretKeySelector(name="mlpipeline-minio-artifact", key="accesskey"),
                            secret_key_secret=V1SecretKeySelector(name="mlpipeline-minio-artifact", key="secretkey"))

def run_notebook_op(name, notebook):
    source = '{}.{}'.format(notebook, 'ipynb')
    result_ipynb = '{}_output.{}'.format(notebook, 'ipynb')
    result_html = '{}_output.{}'.format(notebook, 'html')
    
    op= kfp.dsl.ContainerOp(name=name,
                        image='lresende/notebook-kubeflow-pipeline:dev',
                        artifact_location=notebook_location,
                        arguments=[
                            '--endpoint', 'http://yukked1.fyre.ibm.com:31128',
                            '--user', 'minio',
                            '--password', 'minio123',
                            '--bucket', 'lresende',
                            '--input', source,
                            '--output', result_ipynb,
                            '--output_html', result_html,                            
                        ])
    op.set_image_pull_policy('Always')
    
    return op
    
def demo_pipeline():
    stats_op = run_notebook_op('stats', 'generate-stats')
    contributions_op = run_notebook_op('contributions', 'generate-contributions')
    run_notebook_op('overview', 'overview').after(stats_op, contributions_op)
    
# Compile the new pipeline
kfp.compiler.Compiler().compile(demo_pipeline,'generated-pipelines/pydata_pipeline.tar.gz')

# Upload the compiled pipeline
timestamp = datetime.now().strftime("%m%d%H%M%S")
name='pydata-pipeline'+timestamp
client = kfp.Client(host=url)
client.upload_pipeline('generated-pipelines/pydata_pipeline.tar.gz',pipeline_name=name)
experiment = client.create_experiment(name=name)
run = client.run_pipeline(experiment.id, name, 'generated-pipelines/pydata_pipeline.tar.gz')